# Kerchunk Development (JR)

- Can OOI DAS data be loaded with kerchunk?

In [23]:
import xarray as xr
import dask
from dask.distributed import Client

In [24]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 32,Total memory: 62.80 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34361,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 32
Started: Just now,Total memory: 62.80 GiB
Comm: tcp://127.0.0.1:36783,Total threads: 4
Dashboard: http://127.0.0.1:40639/status,Memory: 7.85 GiB
Nanny: tcp://127.0.0.1:39697,


In [1]:
import fsspec
import kerchunk.hdf
import zarr
import h5py

# Set the account name, container name, and file path
account_name = "dasdata"
container_name = "hdf5"
file_path = "South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T231114Z.h5"

# Set the blob URL using the fsspec AzureBlobFileSystem
url = f"az://{container_name}/{file_path}"
openfile = fsspec.open(url, account_name=account_name)

# Generate dictionary
h5chunks = kerchunk.hdf.SingleHdf5ToZarr(openfile.open(), url)
out = h5chunks.translate()

storage_options = {
    'fo':out,
    'remote_protocol':'az',
    'remote_options':dict(account_name=account_name)
}

z = zarr.open("reference://", storage_options=storage_options)

3472221

In [17]:
da = dask.array.from_zarr(z['Acquisition']['Raw[0]']['RawData'])

In [19]:
dax = xr.DataArray(da)

In [26]:
dax[:1000,:1000].mean().compute()

<xarray.DataArray 'from-zarr-d997e8546162db8203a866bd74b98ca6' ()>
array(-95653.580468)

## Starting from Scratch

In [30]:
import fsspec
import kerchunk.hdf
import zarr
import h5py

In [48]:
# Set the account name, container name, and file path
account_name = "dasdata"
container_name = "hdf5"
file_path = "South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T231114Z.h5"

# Set the blob URL using the fsspec AzureBlobFileSystem
url = f"az://{container_name}/{file_path}"
openfile = fsspec.open(url, account_name=account_name)

# Generate dictionary
h5chunks = kerchunk.hdf.SingleHdf5ToZarr(openfile.open(), url)
out = h5chunks.translate()

In [71]:
print(out['refs']['Acquisition/Raw[0]/RawData/0.0'])

['az://hdf5/South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T231114Z.h5', 262144, 126115]


In [78]:
storage_options = {
    #'fo':out,
    'remote_protocol':'az',
    'remote_options':dict(account_name=account_name)
}
fs = fsspec.open("reference://", fo=out, storage_options=storage_options)

ValueError: unable to connect to account for Must provide either a connection_string or account_name with credentials!!

In [52]:
storage_options = {
    'fo':out,
    'remote_protocol':'az',
    'remote_options':dict(account_name=account_name)
}

z = zarr.open("reference://", storage_options=storage_options)

In [53]:
print(z.tree())

/
 └── Acquisition
     ├── Custom
     └── Raw[0]
         ├── Custom
         │   ├── GpBits (12000,) uint8
         │   ├── GpsStatus (12000,) uint8
         │   ├── PpsOffset (12000,) uint32
         │   └── SampleCount (12000,) int64
         ├── RawData (47500, 12000) int32
         └── RawDataTime (12000,) int64


In [59]:
z['Acquisition']['Raw[0]']['RawData'][0,0]

KeyError: 

In [32]:
hf = h5py.File(openfile.open())

In [39]:
print(hf['Acquisition']['Raw[0]']['RawData'][:5,:5])

[[3472221 3473106 3472831 3471105 3471869]
 [-425498 -424748 -428130 -428440 -429379]
 [-732567 -733165 -733989 -735417 -737383]
 [ 518432  516937  515703  512260  510987]
 [ 645204  644451  642124  639088  637703]]


In [37]:
with open("single_file.json", "w") as outfile:
    json.dump(out, outfile)

## Demo from kerchunk docs

In [1]:
import fsspec

In [3]:
import kerchunk.hdf
import fsspec

urls = ["s3://" + p for p in [
    'noaa-nwm-retro-v2.0-pds/full_physics/2017/201704010000.CHRTOUT_DOMAIN1.comp',
    'noaa-nwm-retro-v2.0-pds/full_physics/2017/201704010100.CHRTOUT_DOMAIN1.comp',
    'noaa-nwm-retro-v2.0-pds/full_physics/2017/201704010200.CHRTOUT_DOMAIN1.comp',
    'noaa-nwm-retro-v2.0-pds/full_physics/2017/201704010300.CHRTOUT_DOMAIN1.comp',
    'noaa-nwm-retro-v2.0-pds/full_physics/2017/201704010400.CHRTOUT_DOMAIN1.comp',
    'noaa-nwm-retro-v2.0-pds/full_physics/2017/201704010500.CHRTOUT_DOMAIN1.comp',
    'noaa-nwm-retro-v2.0-pds/full_physics/2017/201704010600.CHRTOUT_DOMAIN1.comp',
    'noaa-nwm-retro-v2.0-pds/full_physics/2017/201704010700.CHRTOUT_DOMAIN1.comp',
    'noaa-nwm-retro-v2.0-pds/full_physics/2017/201704010800.CHRTOUT_DOMAIN1.comp',
    'noaa-nwm-retro-v2.0-pds/full_physics/2017/201704010900.CHRTOUT_DOMAIN1.comp'
]]
so = dict(
    anon=True, default_fill_cache=False, default_cache_type='first'
)
singles = []
for u in urls:
    with fsspec.open(u, **so) as inf:
        h5chunks = kerchunk.hdf.SingleHdf5ToZarr(inf, u, inline_threshold=100)
        singles.append(h5chunks.translate())

ImportError: Install s3fs to access S3

## Trying with OOI data

In [10]:
import fsspec
import zarr
import kerchunk.hdf
import h5py
import xarray as xr

In [2]:
storage_options = {'account_name':'dasdata'}
url = 'az://hdf5/South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T231114Z.h5'

In [16]:
f = fsspec.open(url, account_name='das_data')
hf = h5py.File(f)

KeyboardInterrupt: 

In [13]:
h5chunks = kerchunk.hdf.SingleHdf5ToZarr(inf, url, inline_threshold=100)

AttributeError: 'AzureBlobFile' object has no attribute '__fspath__'

## Just open the file

In [33]:
import h5py
import fsspec
import zarr
import kerchunk

# Set the account name, container name, and file path
account_name = "dasdata"
container_name = "hdf5"
file_path = "South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T231114Z.h5"

# Set the blob URL using the fsspec AzureBlobFileSystem
blob_url = f"az://{container_name}/{file_path}"
fs = fsspec.filesystem("az", account_name=account_name)
blob_stream = fs.open(blob_url)

f = h5py.File(blob_stream)

In [22]:
def print_h5_objects(item, level=0):
    """
    Recursively prints the names and types of objects in an HDF5 file.
    """
    indent = ' ' * 4 * level
    print(f"{indent}{item.name} ({type(item)})")

    if isinstance(item, h5py.Group):
        for subitem in item.values():
            print_h5_objects(subitem, level + 1)
    elif isinstance(item, h5py.Dataset):
        print(f"{indent}  dtype: {item.dtype}")
        print(f"{indent}  shape: {item.shape}")
        if item.chunks is not None:
            print(f"{indent}  chunked: yes")
            print(f"{indent}  chunks: {item.chunks}")
        else:
            print(f"{indent}  chunked: no")

        for key, value in item.attrs.items():
            print(f"{indent}  {key}: {value}")

In [26]:
f['Acquisition']['Raw[0]']['RawData'][:743, :88*2]

array([[3472221, 3473106, 3472831, ..., 3474074, 3473865, 3475312],
       [-425498, -424748, -428130, ..., -425060, -425510, -424518],
       [-732567, -733165, -733989, ..., -731050, -732228, -731948],
       ...,
       [-183024, -182243, -181445, ..., -189152, -189434, -189755],
       [  -4075,   -3494,   -2887, ...,   -8071,   -8368,   -8651],
       [ -35074,  -34513,  -34001, ...,  -38443,  -38742,  -39041]],
      dtype=int32)

In [23]:
print_h5_objects(f)

/ (<class 'h5py._hl.files.File'>)
    /Acquisition (<class 'h5py._hl.group.Group'>)
        /Acquisition/Custom (<class 'h5py._hl.group.Group'>)
        /Acquisition/Raw[0] (<class 'h5py._hl.group.Group'>)
            /Acquisition/Raw[0]/Custom (<class 'h5py._hl.group.Group'>)
                /Acquisition/Raw[0]/Custom/GpBits (<class 'h5py._hl.dataset.Dataset'>)
                  dtype: uint8
                  shape: (12000,)
                  chunked: yes
                  chunks: (12000,)
                  Description: b'0x01 = Data modified post-acquisition\n0x02 = Data compromised\n'
                /Acquisition/Raw[0]/Custom/GpsStatus (<class 'h5py._hl.dataset.Dataset'>)
                  dtype: uint8
                  shape: (12000,)
                  chunked: yes
                  chunks: (12000,)
                  Description: b'0x01 = GPS Mode\n0x02 = GPS Sync Guaranteed\n0x04 = Time modified post-acquisition\n'
                /Acquisition/Raw[0]/Custom/PpsOffset (<class 'h5p

In [32]:
kerchunk.open

AttributeError: module 'kerchunk' has no attribute 'open'

### GPT attempts

In [29]:
import xarray as xr
import fsspec
import kerchunk

# Set the account name, container name, and file path
account_name = "dasdata"
container_name = "hdf5"
file_path = "South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T231114Z.h5"

# Set the blob URL using the fsspec AzureBlobFileSystem
blob_url = f"az://{container_name}/{file_path}"
fs = fsspec.filesystem("az", account_name=account_name)



In [31]:
fs

In [30]:
# Open the HDF5 file using kerchunk
with fs.open(blob_url) as f:
    ds = kerchunk.open(f)

# Convert to xarray dataset
xr_ds = xr.Dataset.from_dict(ds)


AttributeError: module 'kerchunk' has no attribute 'open'

In [3]:
import fsspec
import zarr
import kerchunk.hdf
import xarray as xr

# Set the account name, container name, and file path
account_name = "dasdata"
container_name = "hdf5"
file_path = "South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T231114Z.h5"

# Set the blob URL using the fsspec AzureBlobFileSystem
blob_url = f"az://{container_name}/{file_path}"
fs = fsspec.filesystem("az", account_name=account_name)


In [7]:

# Convert the HDF5 file to Zarr format using kerchunk.hdf.SingleHdf5ToZarr
with fs.open(blob_url) as f:
    #store = zarr.DirectoryStore("data.zarr")
    h5tozarr = kerchunk.hdf.SingleHdf5ToZarr(fs)
    #h5tozarr(f)

# Open the Zarr dataset using xarray.open_zarr()
#zarr_store = xr.backends.Zarr(store, consolidated=True)
#xr_ds = xr.open_zarr(zarr_store)


TypeError: expected str, bytes or os.PathLike object, not AzureBlobFileSystem

### ChatGPT

In [2]:
import kerchunk
import fsspec

# Set up the Azure Blob Storage filesystem
account_name = 'dasdata'
container_name = 'hdf5'
file_path = 'South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T231114Z.h5'
azure_blob_storage = fsspec.get_filesystem_class('az')(account_name=account_name)

# Open the HDF5 file and convert it to a Zarr array
with azure_blob_storage.open(file_path, 'rb') as hdf5_file:
    converter = kerchunk.SingleHdf5ToZarr(hdf5_file, 'your-zarr-array-name')
    converter.create_array()


ClientAuthenticationError: Server failed to authenticate the request. Please refer to the information in the www-authenticate header.
RequestId:68c92d5b-001e-0043-4c48-4daa63000000
Time:2023-03-02T20:46:38.1343260Z
ErrorCode:NoAuthenticationInformation
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>NoAuthenticationInformation</Code><Message>Server failed to authenticate the request. Please refer to the information in the www-authenticate header.
RequestId:68c92d5b-001e-0043-4c48-4daa63000000
Time:2023-03-02T20:46:38.1343260Z</Message></Error>